In [ ]:
import numpy as np
from scipy.stats import truncnorm
import pandas as pd

In [ ]:
# Define the SIR model parameters
beta = 0.3  # Infection rate
gamma = 0.1  # Recovery rate
population = 1000  # Total population

# Initial state
I0 = 12  # Initial infectious
S0 = population - I0  # Initial susceptible
R0 = 0  # Initial recovered

# Observations
observations = [12, 157, 982, 532, 89]

# Number of particles
n_particles = 100

# Initialize particles
particles = np.zeros((n_particles, 3))
particles[:, 0] = S0
particles[:, 1] = I0
particles[:, 2] = R0

# Function to update particles
def update_particles(particles, beta, gamma, population):
    S, I, R = particles[:, 0], particles[:, 1], particles[:, 2]
    new_infections = np.random.binomial(S.astype(int), 1.0 - np.exp(-beta * I / population))
    new_recoveries = np.random.binomial(I.astype(int), 1.0 - np.exp(-gamma))
    S -= new_infections
    I += new_infections - new_recoveries
    R += new_recoveries
    particles[:, 0], particles[:, 1], particles[:, 2] = S, I, R
    return particles

target_sd = 50.0
def get_importance(particles, mean):
    p_vals = particles[:, 1]
    trunc_vals = -p_vals / target_sd
    return truncnorm.pdf(np.array([mean] * 100), trunc_vals, np.inf, loc=p_vals, scale=target_sd)

# Particle filter implementation
for obs in observations:
    
    # Prediction
    particles = update_particles(particles, beta, gamma, population)

    # Importance
    weights = get_importance(particles, obs) + 1e-10
    norm_weights = weights / sum(weights)
    
    # Resampling
    indices = np.random.choice(range(n_particles), size=n_particles, p=norm_weights)
    particles = particles[indices]
    
    # State estimation
    S_est = np.mean(particles[:, 0])
    I_est = np.mean(particles[:, 1])
    R_est = np.mean(particles[:, 2])
    
    print(f"Observation: {obs}")
    print(f"Estimated Susceptible: {S_est:.2f}")
    print(f"Estimated Infectious: {I_est:.2f}")
    print(f"Estimated Recovered: {R_est:.2f}\n")

In [ ]:
particles